# Project
- 70% des utilisateurs souhaitent plus d'informations à propos de leur destination
- Chercher les meilleures villes en fonction de la météo 
- Proposer des hôtels sur ces villes

In [1]:
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv  

# Récupération des coordonnées GPS des 35 meilleures villes où voyager en France selon One Week In.com

### Test de l'API sur une ville pour voir si l'API renvoie bien quelque chose

In [2]:
url = 'https://data.geopf.fr/geocodage/search'

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3 ',
    "Accept-Language": "fr-FR,fr;q=0.9"
}

payload = {
    'q': 'Mont Saint Michel', 'France'
    'city': 'Mont Saint Michel',
    'format': 'json'
}

response = requests.get(url=url, params=payload, headers=headers)
response.json()


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [5.995538, 45.546586]},
   'properties': {'label': 'Mont Saint Michel 73190 Challes-les-Eaux',
    'score': 0.946500909090909,
    'type': 'locality',
    'importance': 0.41151,
    'id': '73064_B039',
    'banId': 'f67c5810-39e4-4a3d-a92c-89380b8e6ab3',
    'name': 'Mont Saint Michel',
    'postcode': '73190',
    'citycode': '73064',
    'x': 933687.63,
    'y': 6498554.9,
    'city': 'Challes-les-Eaux',
    'context': '73, Savoie, Auvergne-Rhône-Alpes',
    'locality': 'Mont Saint Michel',
    '_type': 'address'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.538198, 47.243963]},
   'properties': {'label': 'Avenue du Mont Saint Michel 44300 Nantes',
    'score': 0.6999590909090908,
    'id': '44109_5785',
    'banId': 'bfc8760b-6503-436a-b194-96398ee4dd18',
    'name': 'Avenue du Mont Saint Michel',
    'postcode': '44300',
    'citycode': '441

### L'API répond, du coup on extrait les coordonnées GPS des 35 villes 

In [4]:
# Liste des villes à interroger
villes = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]



base_url = "https://data.geopf.fr/geocodage/search?"


def get_coordinates(ville):
    
    params = {
        'q': ville,
        'limit': 1,
        'index':'poi'
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
        else:
            return None
    else:
        print(f"Erreur lors de la requête pour {ville}: {response.status_code}")
        return None


results = []
for ville in villes:
    coordinates = get_coordinates(ville)
    if coordinates:
        results.append({'Ville': ville, 'Longitude': coordinates[0], 'Latitude': coordinates[1]})
    else:
        results.append({'Ville': ville, 'Longitude': None, 'Latitude': None})


In [5]:
response.json()['features'][0]['geometry']['coordinates'] #recherche du chemin pour extraire les coordonnées GPS

[5.995538, 45.546586]

### On met les résultats obtenus dans un dataframe

In [ ]:
df = pd.DataFrame(results) # Nous avons bien un dataframe qui contient les 35 villes et leurs coordonnées GPS
df.head()

,Ville,Longitude,Latitude
0,Mont Saint Michel,7.088900,43.849841
1,St Malo,-2.000385,48.653039
2,Bayeux,-0.702368,49.277173
3,Le Havre,0.128407,49.493515
4,Rouen,1.099092,49.443440


In [8]:
df.shape # verification des dimensions savoir si on a bien 35 villes 

(35, 3)

In [9]:
df.isnull().any() #verification des valeurs nulles

Ville        False
Longitude    False
Latitude     False
dtype: bool

### On double check voir si toutes nos coordonnées GPS se situent bien en France

In [11]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", text="Ville",
                        zoom=4, 
                        mapbox_style="open-street-map",
                        title="Verification si mes coordonnées reçues sont bien en France")
fig.update_traces(marker=dict(size=12))
fig.show()

# Maintenant que l'on a les coordonnées GPS pour nos 35 villes on va interroger une autre API pour avoir la météo, notamment la température et les précipications en mm de pluie

On fait un test de l'API sur une ville voir si l'API répond

In [77]:
base_weather_url = 'https://api.open-meteo.com/v1/forecast'
load_dotenv()  # Charger les variables d'environnement depuis le fichier .env


def get_weather(lat, lon):
    #appid = os.getenv('APPID')  # Récupérer l'API key depuis les variables d'environnement
    params = {
        'latitude': lat,
        'longitude': lon,
        'hourly': ["temperature_2m", "rain", "precipitation", "relative_humidity_2m"]
    }
    
    response = requests.get(base_weather_url, params=params)
    print("Status code:", response.status_code)
    print("URL:", response.url)
    print("Response:", response.text)
    return response.json() if response.status_code == 200 else None



In [78]:
# Test with the first city
test_city = df.iloc[1]
weather_data = get_weather(test_city['Longitude'], test_city['Latitude'])
print(weather_data)

Status code: 200
URL: https://api.open-meteo.com/v1/forecast?latitude=-2.000385&longitude=48.653039&hourly=temperature_2m&hourly=rain&hourly=precipitation&hourly=relative_humidity_2m
Response: {"latitude":-2.0,"longitude":48.625,"generationtime_ms":0.2714395523071289,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":0.0,"hourly_units":{"time":"iso8601","temperature_2m":"°C","rain":"mm","precipitation":"mm","relative_humidity_2m":"%"},"hourly":{"time":["2025-05-02T00:00","2025-05-02T01:00","2025-05-02T02:00","2025-05-02T03:00","2025-05-02T04:00","2025-05-02T05:00","2025-05-02T06:00","2025-05-02T07:00","2025-05-02T08:00","2025-05-02T09:00","2025-05-02T10:00","2025-05-02T11:00","2025-05-02T12:00","2025-05-02T13:00","2025-05-02T14:00","2025-05-02T15:00","2025-05-02T16:00","2025-05-02T17:00","2025-05-02T18:00","2025-05-02T19:00","2025-05-02T20:00","2025-05-02T21:00","2025-05-02T22:00","2025-05-02T23:00","2025-05-03T00:00","2025-05-03T01:00","2025-05-03T02:00"

L'API répond, du coup on peut faire la même démarche pour nos 35 villes.\
On boucle sur nos 35 villes souhaitées et on met nos données dans un dataframe pour plus de visibilité

In [79]:
def json_to_df(weather_data, city, lat, lon):
    if weather_data and "hourly" in weather_data:
        df = pd.DataFrame(weather_data["hourly"])
        df["city"] = city
        df["latitude"] = lat
        df["longitude"] = lon
        return df
    else:
        return pd.DataFrame()  # DataFrame vide

# Génération du DataFrame final
df_final = pd.DataFrame()

for city in villes:
    lat, lon = get_coordinates(city)
    if lat is not None and lon is not None:
        weather_data = get_weather(lat, lon)
        df_city = json_to_df(weather_data, city, lat, lon)
        df_final = pd.concat([df_final, df_city], ignore_index=True)
        print(f"{city} OK ({lat}, {lon})")
        

df_final.reset_index(drop=True, inplace=True)
df_final.head()

Status code: 200
URL: https://api.open-meteo.com/v1/forecast?latitude=7.0889&longitude=43.849841&hourly=temperature_2m&hourly=rain&hourly=precipitation&hourly=relative_humidity_2m
Response: {"latitude":7.125,"longitude":43.875,"generationtime_ms":0.16427040100097656,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":900.0,"hourly_units":{"time":"iso8601","temperature_2m":"°C","rain":"mm","precipitation":"mm","relative_humidity_2m":"%"},"hourly":{"time":["2025-05-02T00:00","2025-05-02T01:00","2025-05-02T02:00","2025-05-02T03:00","2025-05-02T04:00","2025-05-02T05:00","2025-05-02T06:00","2025-05-02T07:00","2025-05-02T08:00","2025-05-02T09:00","2025-05-02T10:00","2025-05-02T11:00","2025-05-02T12:00","2025-05-02T13:00","2025-05-02T14:00","2025-05-02T15:00","2025-05-02T16:00","2025-05-02T17:00","2025-05-02T18:00","2025-05-02T19:00","2025-05-02T20:00","2025-05-02T21:00","2025-05-02T22:00","2025-05-02T23:00","2025-05-03T00:00","2025-05-03T01:00","2025-05-03T02:00

,time,temperature_2m,rain,precipitation,relative_humidity_2m,city,latitude,longitude
0,2025-05-02T00:00,22.2,0.0,0.0,86,Mont Saint Michel,7.0889,43.849841
1,2025-05-02T01:00,22.1,0.0,0.0,87,Mont Saint Michel,7.0889,43.849841
2,2025-05-02T02:00,21.8,0.0,0.0,88,Mont Saint Michel,7.0889,43.849841
3,2025-05-02T03:00,21.2,0.0,0.0,92,Mont Saint Michel,7.0889,43.849841
4,2025-05-02T04:00,21.8,0.0,0.0,92,Mont Saint Michel,7.0889,43.849841


In [80]:
df_final.shape # on vérifie les dimensions du dataframe final. On a 5880 lignes, ce qui correspond aux 35 villes et 168 heures de prévisions météo (7 jours * 24 heures) pour chaque ville

(5880, 8)

In [83]:
df_final[df_final['city'] == 'Marseille']

,time,temperature_2m,rain,precipitation,relative_humidity_2m,city,latitude,longitude
3360,2025-05-02T00:00,24.0,0.0,0.0,87,Marseille,5.367037,43.269776
3361,2025-05-02T01:00,23.7,0.0,0.0,87,Marseille,5.367037,43.269776
3362,2025-05-02T02:00,23.4,0.0,0.0,89,Marseille,5.367037,43.269776
3363,2025-05-02T03:00,23.1,0.0,0.0,89,Marseille,5.367037,43.269776
3364,2025-05-02T04:00,24.1,0.0,0.0,86,Marseille,5.367037,43.269776
...,...,...,...,...,...,...,...,...
3523,2025-05-08T19:00,27.3,0.0,0.0,62,Marseille,5.367037,43.269776
3524,2025-05-08T20:00,26.6,0.0,0.0,65,Marseille,5.367037,43.269776
3525,2025-05-08T21:00,26.0,0.0,0.0,69,Marseille,5.367037,43.269776
3526,2025-05-08T22:00,25.5,0.0,0.0,72,Marseille,5.367037,43.269776


In [84]:
df_final['precipitation'].max()

5.4

In [85]:
df_final.isnull().any() #verification des valeurs nulles

time                    False
temperature_2m          False
rain                    False
precipitation           False
relative_humidity_2m    False
city                    False
latitude                False
longitude               False
dtype: bool

In [86]:
df_final['city'].nunique() # verification rapide du nombre de ville unique dans notre dataframe

35

On choisit les 5 meilleures destinations en fonction des températures moyennes et precipitations moyennes